In [4]:
from pwn import *
from egcd import egcd
from hashlib import sha256
from Crypto.Util import number

context.log_level = 'critical'
# ctx = process(["python3", "chall.py"])
ctx = remote("52.59.124.14", 5026)

begin = ctx.recvuntil(b"> ").split(b"\n")
n = int(begin[0].decode().split(" ")[2])
a = int(begin[1].decode().split(" ")[2])
e = int(begin[2].decode().split(" ")[2])

ctx.sendline(b"1")
answer = ctx.recvuntil(b"> ").split(b"\n")
c1 = bytes.fromhex(answer[0].decode().split(" ")[1])
s1 = int(answer[1].decode().split(" ")[1])
h1 = number.bytes_to_long(sha256(c1).digest())


ctx.sendline(b"1")
answer = ctx.recvuntil(b"> ").split(b"\n")
c2 = bytes.fromhex(answer[0].decode().split(" ")[1])
s2 = int(answer[1].decode().split(" ")[1])
h2 = number.bytes_to_long(sha256(c2).digest())

gcd, u, v = egcd(h1, h2)
assert gcd == 1 # Just restart if it's the case
print(u*h1 + v*h2)

ctx.sendline(b"2")
answer = ctx.recvuntil(b"Signature: ").split(b"\n")
c3 = bytes.fromhex(answer[0].decode().split(" ")[1])
h3 = number.bytes_to_long(sha256(c3).digest())

to_send = pow(pow(s1, u, n)*pow(s2, v, n), h3, n)

ctx.sendline(str(to_send).encode())
print(ctx.recvline().decode())


1
ENO{ha5h_ev3ryth1ng_th3y_s4id_s00o_s3cur3}

